In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

dataset_path = #r'D:\Accenture_Project\Plastic Pollution.v4i.tensorflow\Directory'

datagen = ImageDataGenerator(rescale=1./255,
                             validation_split=0.3,
                             horizontal_flip=True,
                             rotation_range=45)

train_data = datagen.flow_from_directory(os.path.join(dataset_path),
                                         target_size=(224, 224),
                                         batch_size=100,
                                         subset='training')

val_data = datagen.flow_from_directory(os.path.join(dataset_path),
                                       target_size=(224, 224),
                                       batch_size=100,
                                       subset='validation')

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(100, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(100, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(train_data, validation_data=val_data, callbacks=[early_stopping], epochs=10)

test_data = datagen.flow_from_directory(os.path.join(dataset_path, 'test'),
                                        target_size=(224, 224),
                                        batch_size=100,
                                        shuffle=False)

loss, accuracy = model.evaluate(test_data)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

from tensorflow.keras.preprocessing.image import load_img

image_path = #r'D:\Accenture_Project\Plastic Pollution.v4i.tensorflow\test\Plastic-Pollution_original_plastic--2--jpg_521af4bb-4a08-408b-a220-5f3a15195205_jpg.rf.bb471805d3216ad1de16a9cb9c474903.jpg'
image = load_img(image_path, target_size=(224, 224))
image_array = tf.keras.preprocessing.image.img_to_array(image)
image_array = image_array / 255.0

image_array = image_array.reshape((1, 224, 224, 3))

predictions = model.predict(image_array)
confidence = predictions[0][0]

class_label = 'plastic waste' if confidence >= 0.50 else 'non-plastic waste'
print('Prediction:', class_label)
print('Confidence:', confidence)
